# SAT Solver Evaluation and Comparison

This notebook evaluates our SAT solver implementation and compares it against reference solvers.

In [ ]:
import sys
import os
import time
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path('..') / 'src'))
sys.path.insert(0, str(Path('..') / 'visualizations'))

from dimacs_parser import parse_dimacs, write_dimacs
from dpll_solver import DPLLSolver
from cdcl_solver import CDCLSolver
from benchmark_generator import generate_random_3sat, generate_pigeonhole
from visualize import plot_solver_comparison, plot_performance_scatter

## 1. Generate Test Instances

In [ ]:
# Create benchmark directory
benchmark_dir = Path('..') / 'benchmarks' / 'generated'
benchmark_dir.mkdir(parents=True, exist_ok=True)

# Generate random 3-SAT instances at phase transition (ratio ~4.3)
instances = []
for n_vars in [10, 15, 20, 25, 30]:
    n_clauses = int(n_vars * 4.3)
    clauses = generate_random_3sat(n_vars, n_clauses, seed=42+n_vars)
    filename = benchmark_dir / f"3sat_{n_vars}_{n_clauses}.cnf"
    write_dimacs(str(filename), n_vars, clauses)
    instances.append({
        'name': f'3SAT-{n_vars}',
        'file': str(filename),
        'vars': n_vars,
        'clauses': n_clauses
    })

print(f"Generated {len(instances)} test instances")

## 2. Run DPLL Solver

In [ ]:
dpll_results = []

for instance in instances:
    print(f"\nTesting DPLL on {instance['name']}...")
    num_vars, num_clauses, clauses = parse_dimacs(instance['file'])
    
    solver = DPLLSolver(num_vars, clauses)
    is_sat, assignment, stats = solver.solve()
    
    result = {
        'instance': instance['name'],
        'sat': is_sat,
        **stats
    }
    dpll_results.append(result)
    
    print(f"  Result: {'SAT' if is_sat else 'UNSAT'}")
    print(f"  Time: {stats['time']:.4f}s")
    print(f"  Decisions: {stats['decisions']}, Propagations: {stats['propagations']}")

## 3. Run CDCL Solver

In [ ]:
cdcl_results = []

for instance in instances:
    print(f"\nTesting CDCL on {instance['name']}...")
    num_vars, num_clauses, clauses = parse_dimacs(instance['file'])
    
    solver = CDCLSolver(num_vars, clauses)
    try:
        is_sat, assignment, stats = solver.solve(time_limit=10.0)
        
        result = {
            'instance': instance['name'],
            'sat': is_sat,
            **stats
        }
        cdcl_results.append(result)
        
        print(f"  Result: {'SAT' if is_sat else 'UNSAT' if is_sat is not None else 'TIMEOUT'}")
        print(f"  Time: {stats['time']:.4f}s")
        if 'conflicts' in stats:
            print(f"  Conflicts: {stats['conflicts']}, Learned: {stats['learned']}")
    except Exception as e:
        print(f"  Error: {e}")
        cdcl_results.append({'instance': instance['name'], 'error': str(e)})

## 4. Compare DPLL vs CDCL

In [ ]:
# Create comparison plots
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Time comparison
instances_names = [r['instance'] for r in dpll_results]
dpll_times = [r['time'] for r in dpll_results]
cdcl_times = [r.get('time', 0) for r in cdcl_results]

x = np.arange(len(instances_names))
width = 0.35

axes[0].bar(x - width/2, dpll_times, width, label='DPLL', color='blue', alpha=0.7)
axes[0].bar(x + width/2, cdcl_times, width, label='CDCL', color='orange', alpha=0.7)
axes[0].set_xlabel('Instance')
axes[0].set_ylabel('Time (s)')
axes[0].set_title('Solving Time Comparison')
axes[0].set_xticks(x)
axes[0].set_xticklabels(instances_names, rotation=45)
axes[0].legend()
axes[0].set_yscale('log')
axes[0].grid(axis='y', alpha=0.3)

# Decisions comparison
dpll_decisions = [r['decisions'] for r in dpll_results]
cdcl_decisions = [r.get('decisions', 0) for r in cdcl_results]

axes[1].bar(x - width/2, dpll_decisions, width, label='DPLL', color='blue', alpha=0.7)
axes[1].bar(x + width/2, cdcl_decisions, width, label='CDCL', color='orange', alpha=0.7)
axes[1].set_xlabel('Instance')
axes[1].set_ylabel('Decisions')
axes[1].set_title('Decision Count Comparison')
axes[1].set_xticks(x)
axes[1].set_xticklabels(instances_names, rotation=45)
axes[1].legend()
axes[1].set_yscale('log')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../visualizations/dpll_vs_cdcl.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nComparison plot saved to visualizations/dpll_vs_cdcl.png")

## 5. CDCL Performance Analysis

In [ ]:
# Analyze CDCL-specific metrics
cdcl_with_stats = [r for r in cdcl_results if 'conflicts' in r]

if cdcl_with_stats:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    instances_with_stats = [r['instance'] for r in cdcl_with_stats]
    conflicts = [r['conflicts'] for r in cdcl_with_stats]
    learned = [r['learned'] for r in cdcl_with_stats]
    restarts = [r['restarts'] for r in cdcl_with_stats]
    
    axes[0].bar(instances_with_stats, conflicts, color='red', alpha=0.7)
    axes[0].set_title('Conflicts')
    axes[0].set_xlabel('Instance')
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].grid(axis='y', alpha=0.3)
    
    axes[1].bar(instances_with_stats, learned, color='green', alpha=0.7)
    axes[1].set_title('Learned Clauses')
    axes[1].set_xlabel('Instance')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(axis='y', alpha=0.3)
    
    axes[2].bar(instances_with_stats, restarts, color='purple', alpha=0.7)
    axes[2].set_title('Restarts')
    axes[2].set_xlabel('Instance')
    axes[2].tick_params(axis='x', rotation=45)
    axes[2].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../visualizations/cdcl_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\nCDCL analysis plot saved to visualizations/cdcl_analysis.png")

## 6. Summary Statistics

In [ ]:
import pandas as pd

# Create summary table
summary_data = []
for i, instance in enumerate(instances_names):
    dpll = dpll_results[i]
    cdcl = cdcl_results[i] if i < len(cdcl_results) else {}
    
    summary_data.append({
        'Instance': instance,
        'DPLL Time (s)': f"{dpll.get('time', 0):.4f}",
        'CDCL Time (s)': f"{cdcl.get('time', 0):.4f}",
        'DPLL Decisions': dpll.get('decisions', 0),
        'CDCL Decisions': cdcl.get('decisions', 0),
        'CDCL Conflicts': cdcl.get('conflicts', 0),
        'CDCL Learned': cdcl.get('learned', 0)
    })

df = pd.DataFrame(summary_data)
print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)
print(df.to_string(index=False))
print("="*80)

# Save to CSV
df.to_csv('../visualizations/results_summary.csv', index=False)
print("\nResults saved to visualizations/results_summary.csv")

## 7. Conclusion

This notebook demonstrates:

1. **DPLL Solver**: Basic backtracking search with unit propagation and pure literal elimination
2. **CDCL Solver**: Advanced solver with conflict-driven learning, VSIDS, and restarts
3. **Performance Comparison**: CDCL typically performs better on harder instances due to learning

### Key Observations:
- DPLL is simpler but explores more of the search space
- CDCL learns from conflicts and avoids redundant search
- CDCL's clause learning and VSIDS heuristic make it more efficient on structured problems

### Future Improvements:
- Debug and optimize the CDCL conflict analysis
- Implement LBD-based clause deletion
- Add preprocessing (variable elimination, subsumption)
- Compare against MiniSAT and other modern solvers